In [1]:
!pip install flask flask-cors pyngrok


In [2]:
from pyngrok import ngrok

# Authtoken personal (puedes revocarlo en el dashboard si quieres en el futuro)
ngrok.set_auth_token("2w5n1KQhLka9SlpuV3aXRahjwUF_2ou5PSU8UFMLEEUay91Jr")



In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
from PIL import Image, ImageOps
import io
import numpy as np

app = Flask(__name__)
CORS(app)

@app.route("/procesar", methods=["POST"])
def procesar_imagen():
    try:
        data = request.get_json()
        imagen_base64 = data['imagen'].split(',')[1]

        # Decodificar imagen
        imagen_bytes = base64.b64decode(imagen_base64)
        imagen = Image.open(io.BytesIO(imagen_bytes)).convert('L')  # escala de grises

        # Detectar trazo
        arr = np.array(imagen)
        threshold = 240
        mask = arr < threshold

        if not mask.any():
            return jsonify({
                "mensaje": "No se detectó dibujo.",
                "imagen_sprite": ""
            })

        coords = np.argwhere(mask)
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0) + 1

        # Recorte y centrado
        imagen_recortada = imagen.crop((x0, y0, x1, y1)).convert("RGB")
        imagen_centrada = Image.new("RGB", (140, 140), (255, 255, 255))
        imagen_redimensionada = ImageOps.contain(imagen_recortada, (140, 140))
        offset = ((140 - imagen_redimensionada.width) // 2, (140 - imagen_redimensionada.height) // 2)
        imagen_centrada.paste(imagen_redimensionada, offset)

        # 🔥 Convertir a RGBA y hacer fondo blanco transparente
        imagen_rgba = imagen_centrada.convert("RGBA")
        datas = imagen_rgba.getdata()
        nueva_data = []
        for item in datas:
            if item[0] > 240 and item[1] > 240 and item[2] > 240:
                nueva_data.append((255, 255, 255, 0))  # transparencia
            else:
                nueva_data.append(item)
        imagen_rgba.putdata(nueva_data)

        # Convertir a base64
        buffered = io.BytesIO()
        imagen_rgba.save(buffered, format="PNG")
        nueva_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        nueva_base64 = f"data:image/png;base64,{nueva_base64}"

        return jsonify({
            "mensaje": "¡Sprite procesado con fondo transparente!",
            "imagen_sprite": nueva_base64
        })

    except Exception as e:
        return jsonify({
            "mensaje": f"Error: {str(e)}",
            "imagen_sprite": ""
        })


In [6]:
public_url = ngrok.connect(5000)
print("✅ URL pública del backend:", public_url)
app.run(port=5000)



✅ URL pública del backend: NgrokTunnel: "https://ad2c-34-90-232-12.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:38:30] "OPTIONS /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:38:30] "POST /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:39:17] "OPTIONS /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:39:17] "POST /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:41:38] "OPTIONS /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:41:38] "POST /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:43:22] "OPTIONS /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:43:23] "POST /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 02:44:35] "OPTIONS /procesar HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Ap